In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import numpy as np
import json
import plotly.plotly as py
import plotly.graph_objs as graph_objs

%matplotlib inline

In [2]:
df = pd.read_csv('Population_Colorado.csv')

with open('colorado_counties.geojson') as f:
    counties = json.load(f)

In [3]:
d = {}
for x in range(len(counties['features'])):
    name = counties['features'][x]['properties']['name'][:-11]
    if name in df['county'].unique():
        d[name] = counties['features'][x]
    else:
        print 'not in: ', name

ser = pd.Series(d.values(), index = d.keys())
ser.name = 'coordinates'

full_df = df.groupby(['county', 'year'], as_index=False).sum()
full_df.drop(['fipsCode', 'age'], inplace = True, axis=1)

full_df = full_df.join(ser, on='county')

In [4]:
full_df.head()

,county,year,malePopulation,femalePopulation,totalPopulation,coordinates
0,Adams,1990,131902,133807,265709,"{u'geometry': {u'type': u'MultiPolygon', u'coo..."
1,Adams,1991,136193,137427,273620,"{u'geometry': {u'type': u'MultiPolygon', u'coo..."
2,Adams,1992,140356,141030,281386,"{u'geometry': {u'type': u'MultiPolygon', u'coo..."
3,Adams,1993,145351,145431,290782,"{u'geometry': {u'type': u'MultiPolygon', u'coo..."
4,Adams,1994,150673,150120,300793,"{u'geometry': {u'type': u'MultiPolygon', u'coo..."


In [5]:
# colors from
# https://gka.github.io/palettes/#colors=lightyellow,orange,deeppink,darkred|steps=7|bez=1|coL=1

colors = ['#ffffe0','#fffddb','#fffad7','#fff7d1','#fff5cd','#fff2c8',
          '#fff0c4','#ffedbf','#ffebba','#ffe9b7','#ffe5b2','#ffe3af',
          '#ffe0ab','#ffdda7','#ffdba4','#ffd9a0','#ffd69c','#ffd399',
          '#ffd196','#ffcd93','#ffca90','#ffc88d','#ffc58a','#ffc288',
          '#ffbf86','#ffbd83','#ffb981','#ffb67f','#ffb47d','#ffb17b',
          '#ffad79','#ffaa77','#ffa775','#ffa474','#ffa172','#ff9e70',
          '#ff9b6f','#ff986e','#ff956c','#fe916b','#fe8f6a','#fd8b69',
          '#fc8868','#fb8567','#fa8266','#f98065','#f87d64','#f77a63',
          '#f67862','#f57562','#f37261','#f37060','#f16c5f','#f0695e',
          '#ee665d','#ed645c','#ec615b','#ea5e5b','#e85b59','#e75859',
          '#e55658','#e45356','#e35056','#e14d54','#df4a53','#dd4852',
          '#db4551','#d9434f','#d8404e','#d53d4d','#d43b4b','#d2384a',
          '#cf3548','#cd3346','#cc3045','#ca2e43','#c72b42','#c52940',
          '#c2263d','#c0233c','#be213a','#bb1e37','#ba1c35','#b71933',
          '#b41731','#b2152e','#b0122c','#ac1029','#aa0e27','#a70b24',
          '#a40921','#a2071f','#a0051c','#9d0419','#990215','#970212',
          '#94010e','#91000a','#8e0006','#8b0000', '#8b0000']

scl = dict(zip(np.arange(0, 101, 1), colors))

In [6]:
def get_scl(obj):
    frac = obj / 10000
    return scl[frac]
    
full_df['color'] = full_df['totalPopulation'].apply(get_scl)

In [10]:
df_2015 = full_df[full_df['year'] == 2015]

In [12]:
layers_ls = []
for x in df_2015.index:
    item_dict = dict(sourcetype = 'geojson',
                     source = df_2015.ix[x]['coordinates'],
                     type = 'fill',
                     color = df_2015.ix[x]['color'])
    layers_ls.append(item_dict)

In [13]:
colorscl = [[i * .01, v] for i,v in enumerate(scl.values())]

In [14]:
mapbox_access_token = os.environ['MAPBOX_AT']

In [ ]:
nums = [k/float(100) for k in scl.keys()]

data = graph_objs.Data([
            graph_objs.Scattermapbox(
                    lat = [0],
                    lon = [0],
                    marker = graph_objs.Marker(
                            size=9,
                            cmax=100,
                            cmin=0,
                            colorscale = colorscl,
                            showscale = True,
                            autocolorscale=False,
                            color=range(0,101),
                            colorbar=graph_objs.ColorBar(
                                                        yanchor='middle',
                                                        xanchor='center',
#                                                         title='population (x 10000)',
                                                        titlefont = dict(size = 15),
                                                        titleside = 'top',
                                                        y = .5,
                                                        x = .96,
                                                        len = .89
                                                        ),
                                                ),
                    mode = 'markers',
                                     )
                                    
                     ])
            
layout = graph_objs.Layout(
    title = 'Colorado Population (x 10,000)',
    height=1050,
    width=800,
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        layers= layers_ls,
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=39.03,
            lon=-105.7
        ),
        pitch=0,
        zoom=5.5,
        style='light'
    ),
)

fig = dict(data = data, layout=layout)
fn = 'test.jpeg'
save_fn = 'images/{}'.format(fn)
h = 575
w = 750
py.image.save_as(fig, filename=save_fn, width = w, height= h)
py.image.ishow(fig, width = w, height= h)

In [ ]:
from IPython.display import Image

Image('images/test.jpeg')